<center>
<h1><b>Segmenting and Clustering Neighborhoods in Toronto</b><h1>
<h3><b>Week 2 Assignment</b></h3>
<br>
<h3>IBM Data Science Professional Certificate<h3>
<h4>Course 10 - Applied Data Science Capstone</h4>
<br>
<h4>Submitted by Pratham Sharma</h4>
</center>

This is the Jupyter Notebook for the assignment for week 2 of the course "Applied Data Science Capstone" of the IBM Data Science Professional Certificate specialization on Coursera.

The task is to scrape data for neighborhoods in Toronto, CA from Wikipedia. The data is then transformed into a DataFrame using BeautifulSoup and Pandas.

# Libraries & Packages

All required packages and libraries are imported in the following code cell.

In [1]:
# For getting Wikipedia page from URl hrough get request
import requests

# For scraping information from the HTML source
from bs4 import BeautifulSoup

# To create the DataFrame for neighborhood data
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# To get latitude and longitude values for given address
!pip install geopy==2.1.0
from geopy.geocoders import Nominatim

print("Libraries Imported!")

Libraries Imported!


# Web Scraping

The following code cell includes the code for scraping data from the Wikipedia page titled 'List of postal codes of Canada: M'. The scraped web page is transformed into a Pandas DataFrame.

The DataFrame consist of three columns - 'Postal Code', 'Borough' and 'Neighborhood'.

Finally, the number of records (rows) in the DataFrame is displayed.

In [2]:
# Get the HTML source
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

# Initialise empty list to store table contents
table_contents = []

# Scrape all information in the HTML 'table' tag
table = soup.find('table')
# Scrape information in table rows i.e., HTML 'td' tag
for row in table.findAll('td'):
    # Store record of DataFrame
    cell = {}
    # Drop record if borough is 'Not assigned'
    if row.span.text=='Not assigned':
        pass
    else:
        # Add PostalCode, Borough and Neighborhoogd to record
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# Create Pandas DataFrame
canada_df = pd.DataFrame(table_contents)

# Replace Borough values with appropriate name
canada_df['Borough'] = canada_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                     'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                     'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

# Diaplay first 10 records
canada_df.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [3]:
# Display umber of rows
print('Number of rows in DataFrame: {}'.format(canada_df.shape[0]))

Number of rows in DataFrame: 103


# Coordinates for Neighborhodds

Finally, here I have added the columns for latitude and longitude values by joining the DataFrame created above with the Geospatial dataset provided.

In [4]:
geospatial_data = pd.read_csv('Geospatial_Coordinates.csv')

canada_df = canada_df.join(geospatial_data.set_index('Postal Code'), on='Postal Code')
canada_df.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
